In [1]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [2]:
# read in with pandas
winter1 = pd.read_csv ('pvd20-21data.txt')
winter2 = pd.read_csv ('pvd21-22data.txt')
winter3 = pd.read_csv ('pvd22-23data.txt')

In [3]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [4]:
# mask out days without precipitation
codes = ['GR','GS','IC','PL','PY','RA','SG','SH','SN','SQ','TS','UP']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [5]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_2512\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_2512\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_2512\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [6]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_2512\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_2512\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_2512\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [7]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 51]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 51]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 51]

In [8]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [9]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [46]:
pvd = LocationInfo(
    'Providence', 'Rhode Island', 
    latitude=41.8240,  
    longitude=-71.4128,
    timezone='America/Rhode_Island'
)

pvd

LocationInfo(name='Providence', region='Rhode Island', timezone='America/Rhode_Island', latitude=41.824, longitude=-71.4128)

In [48]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(pvd.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [50]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(pvd.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [52]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(pvd.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [54]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 06:54:22  2020-12-01 16:15:14  2020-12-01
1   2020-12-02 06:55:24  2020-12-02 16:15:00  2020-12-02
2   2020-12-03 06:56:24  2020-12-03 16:14:47  2020-12-03
3   2020-12-04 06:57:24  2020-12-04 16:14:38  2020-12-04
4   2020-12-05 06:58:22  2020-12-05 16:14:30  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:28:07  2021-02-24 17:30:03  2021-02-24
86  2021-02-25 06:26:35  2021-02-25 17:31:16  2021-02-25
87  2021-02-26 06:25:03  2021-02-26 17:32:28  2021-02-26
88  2021-02-27 06:23:30  2021-02-27 17:33:40  2021-02-27
89  2021-02-28 06:21:56  2021-02-28 17:34:52  2021-02-28

[90 rows x 3 columns]


In [56]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 06:54:06  2021-12-01 16:15:18  2021-12-01
1   2021-12-02 06:55:09  2021-12-02 16:15:03  2021-12-02
2   2021-12-03 06:56:10  2021-12-03 16:14:50  2021-12-03
3   2021-12-04 06:57:10  2021-12-04 16:14:39  2021-12-04
4   2021-12-05 06:58:08  2021-12-05 16:14:31  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:28:29  2022-02-24 17:29:46  2022-02-24
86  2022-02-25 06:26:58  2022-02-25 17:30:58  2022-02-25
87  2022-02-26 06:25:25  2022-02-26 17:32:11  2022-02-26
88  2022-02-27 06:23:52  2022-02-27 17:33:23  2022-02-27
89  2022-02-28 06:22:18  2022-02-28 17:34:35  2022-02-28

[90 rows x 3 columns]


In [58]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 06:53:51  2022-12-01 16:15:22  2022-12-01
1   2022-12-02 06:54:54  2022-12-02 16:15:06  2022-12-02
2   2022-12-03 06:55:55  2022-12-03 16:14:52  2022-12-03
3   2022-12-04 06:56:55  2022-12-04 16:14:41  2022-12-04
4   2022-12-05 06:57:54  2022-12-05 16:14:33  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:28:51  2023-02-24 17:29:28  2023-02-24
86  2023-02-25 06:27:20  2023-02-25 17:30:41  2023-02-25
87  2023-02-26 06:25:48  2023-02-26 17:31:53  2023-02-26
88  2023-02-27 06:24:15  2023-02-27 17:33:05  2023-02-27
89  2023-02-28 06:22:41  2023-02-28 17:34:17  2023-02-28

[90 rows x 3 columns]


In [60]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

In [62]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,PVD,2020-12-01 00:51:00,2.29,+RA BR,2020-12-01,2020-12-01 06:54:22,2020-12-01 16:15:14
1,PVD,2020-12-01 01:51:00,7.87,-RA BR,2020-12-01,2020-12-01 06:54:22,2020-12-01 16:15:14
2,PVD,2020-12-01 02:51:00,3.81,-RA BR,2020-12-01,2020-12-01 06:54:22,2020-12-01 16:15:14
3,PVD,2020-12-01 03:51:00,4.83,-RA BR,2020-12-01,2020-12-01 06:54:22,2020-12-01 16:15:14
4,PVD,2020-12-01 04:51:00,13.72,+RA BR,2020-12-01,2020-12-01 06:54:22,2020-12-01 16:15:14
...,...,...,...,...,...,...,...
271,PVD,2021-02-27 16:51:00,1.78,RA BR,2021-02-27,2021-02-27 06:23:30,2021-02-27 17:33:40
272,PVD,2021-02-27 17:51:00,3.56,RA BR,2021-02-27,2021-02-27 06:23:30,2021-02-27 17:33:40
273,PVD,2021-02-27 18:51:00,4.06,RA BR,2021-02-27,2021-02-27 06:23:30,2021-02-27 17:33:40
274,PVD,2021-02-27 19:51:00,3.30,RA BR,2021-02-27,2021-02-27 06:23:30,2021-02-27 17:33:40


In [64]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,PVD,2021-12-02 11:51:00,0.0001,-RA,2021-12-02,2021-12-02 06:55:09,2021-12-02 16:15:03
1,PVD,2021-12-02 23:51:00,0.0001,-RA,2021-12-02,2021-12-02 06:55:09,2021-12-02 16:15:03
2,PVD,2021-12-06 09:51:00,0.0001,-RA,2021-12-06,2021-12-06 06:59:06,2021-12-06 16:14:26
3,PVD,2021-12-06 10:51:00,2.7900,-RA,2021-12-06,2021-12-06 06:59:06,2021-12-06 16:14:26
4,PVD,2021-12-06 13:51:00,0.0001,-RA BR,2021-12-06,2021-12-06 06:59:06,2021-12-06 16:14:26
...,...,...,...,...,...,...,...
296,PVD,2022-02-25 17:51:00,0.2500,-RA BR,2022-02-25,2022-02-25 06:26:58,2022-02-25 17:30:58
297,PVD,2022-02-25 18:51:00,0.7600,-RAPL BR,2022-02-25,2022-02-25 06:26:58,2022-02-25 17:30:58
298,PVD,2022-02-25 19:51:00,0.2500,-RASN,2022-02-25,2022-02-25 06:26:58,2022-02-25 17:30:58
299,PVD,2022-02-25 20:51:00,0.0001,-SN BR,2022-02-25,2022-02-25 06:26:58,2022-02-25 17:30:58


In [66]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,PVD,2022-12-01 00:51:00,4.0600,-RA BR,2022-12-01,2022-12-01 06:53:51,2022-12-01 16:15:22
1,PVD,2022-12-01 01:51:00,0.7600,-RA,2022-12-01,2022-12-01 06:53:51,2022-12-01 16:15:22
2,PVD,2022-12-03 13:51:00,0.0001,-RA,2022-12-03,2022-12-03 06:55:55,2022-12-03 16:14:52
3,PVD,2022-12-03 15:51:00,0.2500,-RA,2022-12-03,2022-12-03 06:55:55,2022-12-03 16:14:52
4,PVD,2022-12-03 16:51:00,0.7600,-RA BR,2022-12-03,2022-12-03 06:55:55,2022-12-03 16:14:52
...,...,...,...,...,...,...,...
303,PVD,2023-02-26 01:51:00,0.0001,-SN,2023-02-26,2023-02-26 06:25:48,2023-02-26 17:31:53
304,PVD,2023-02-26 02:51:00,0.0001,-SN,2023-02-26,2023-02-26 06:25:48,2023-02-26 17:31:53
305,PVD,2023-02-26 03:51:00,0.0001,-SN,2023-02-26,2023-02-26 06:25:48,2023-02-26 17:31:53
306,PVD,2023-02-27 10:51:00,0.0001,-SN,2023-02-27,2023-02-27 06:24:15,2023-02-27 17:33:05


In [68]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
6,PVD,2020-12-01 07:51:00,0.0001,-RA BR,2020-12-01,2020-12-01 06:54:22,2020-12-01 16:15:14
13,PVD,2020-12-05 07:51:00,0.0001,-RA,2020-12-05,2020-12-05 06:58:22,2020-12-05 16:14:30
14,PVD,2020-12-05 08:51:00,0.2500,-RA,2020-12-05,2020-12-05 06:58:22,2020-12-05 16:14:30
15,PVD,2020-12-05 09:51:00,1.5200,RA,2020-12-05,2020-12-05 06:58:22,2020-12-05 16:14:30
16,PVD,2020-12-05 10:51:00,3.5600,RA BR,2020-12-05,2020-12-05 06:58:22,2020-12-05 16:14:30
...,...,...,...,...,...,...,...
267,PVD,2021-02-27 11:51:00,0.0001,-RA,2021-02-27,2021-02-27 06:23:30,2021-02-27 17:33:40
268,PVD,2021-02-27 13:51:00,0.2500,-RA,2021-02-27,2021-02-27 06:23:30,2021-02-27 17:33:40
269,PVD,2021-02-27 14:51:00,0.2500,-RA BR,2021-02-27,2021-02-27 06:23:30,2021-02-27 17:33:40
270,PVD,2021-02-27 15:51:00,1.2700,RA BR,2021-02-27,2021-02-27 06:23:30,2021-02-27 17:33:40


In [70]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_2512\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_2512\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_2512\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [72]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
14,PVD,2020-12-05 08:51:00,0.2500,-RA,2020-12-05,2020-12-05 06:58:22,2020-12-05 16:14:30,2020-12-05 07:58:22,2020-12-05 15:14:30
15,PVD,2020-12-05 09:51:00,1.5200,RA,2020-12-05,2020-12-05 06:58:22,2020-12-05 16:14:30,2020-12-05 07:58:22,2020-12-05 15:14:30
16,PVD,2020-12-05 10:51:00,3.5600,RA BR,2020-12-05,2020-12-05 06:58:22,2020-12-05 16:14:30,2020-12-05 07:58:22,2020-12-05 15:14:30
17,PVD,2020-12-05 11:51:00,4.3200,-RA BR,2020-12-05,2020-12-05 06:58:22,2020-12-05 16:14:30,2020-12-05 07:58:22,2020-12-05 15:14:30
18,PVD,2020-12-05 12:51:00,0.7600,RA BR,2020-12-05,2020-12-05 06:58:22,2020-12-05 16:14:30,2020-12-05 07:58:22,2020-12-05 15:14:30
...,...,...,...,...,...,...,...,...,...
257,PVD,2021-02-20 10:51:00,0.0001,-SN,2021-02-20,2021-02-20 06:34:05,2021-02-20 17:25:10,2021-02-20 07:34:05,2021-02-20 16:25:10
267,PVD,2021-02-27 11:51:00,0.0001,-RA,2021-02-27,2021-02-27 06:23:30,2021-02-27 17:33:40,2021-02-27 07:23:30,2021-02-27 16:33:40
268,PVD,2021-02-27 13:51:00,0.2500,-RA,2021-02-27,2021-02-27 06:23:30,2021-02-27 17:33:40,2021-02-27 07:23:30,2021-02-27 16:33:40
269,PVD,2021-02-27 14:51:00,0.2500,-RA BR,2021-02-27,2021-02-27 06:23:30,2021-02-27 17:33:40,2021-02-27 07:23:30,2021-02-27 16:33:40


In [74]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [76]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3

In [78]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 1.4837022222222223
2021-2022: 1.0892514563106797
2022-2023: 0.982420238095238


In [80]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,PVD,2020-12-01 00:51:00,2.2900,+RA BR,2020-12-01,2020-12-01 06:54:22,2020-12-01 16:15:14
1,PVD,2020-12-01 01:51:00,7.8700,-RA BR,2020-12-01,2020-12-01 06:54:22,2020-12-01 16:15:14
2,PVD,2020-12-01 02:51:00,3.8100,-RA BR,2020-12-01,2020-12-01 06:54:22,2020-12-01 16:15:14
3,PVD,2020-12-01 03:51:00,4.8300,-RA BR,2020-12-01,2020-12-01 06:54:22,2020-12-01 16:15:14
4,PVD,2020-12-01 04:51:00,13.7200,+RA BR,2020-12-01,2020-12-01 06:54:22,2020-12-01 16:15:14
...,...,...,...,...,...,...,...
266,PVD,2021-02-23 22:51:00,0.0001,-RA,2021-02-23,2021-02-23 06:29:38,2021-02-23 17:28:50
272,PVD,2021-02-27 17:51:00,3.5600,RA BR,2021-02-27,2021-02-27 06:23:30,2021-02-27 17:33:40
273,PVD,2021-02-27 18:51:00,4.0600,RA BR,2021-02-27,2021-02-27 06:23:30,2021-02-27 17:33:40
274,PVD,2021-02-27 19:51:00,3.3000,RA BR,2021-02-27,2021-02-27 06:23:30,2021-02-27 17:33:40


In [82]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_2512\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_2512\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_2512\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [84]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [86]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [31]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 1.322820930232558
2021-2022: 0.5606313432835822
2022-2023: 0.8458133333333333
